# `auto_LiRPA` Quick start tutorial (AAAI 2022)

auto_LiRPA is a library for automatically deriving and computing bounds with linear relaxation based perturbation analysis (LiRPA) (e.g. CROWN and DeepPoly) for neural networks. LiRPA algorithms can provide guaranteed upper and lower bounds for a neural network function with perturbed inputs. These bounds are represented as linear functions with respect to the variable under perturbation. LiRPA has become an important tool in robustness verification and certified adversarial defense, and can become an useful tool for many other tasks as well.

Our algorithm generalizes existing LiRPA algorithms for feed-forward neural networks to a graph algorithm on general computational graphs. We can compute LiRPA bounds on a computational graph defined by PyTorch, without any manual derivation. Our implementation is also automatically differentiable, allowing optimizing network parameters to shape the bounds into certain specifications (e.g., certified defense).

## Installation & Imports
We first install the auto_LiRPA library using pip. Note that our library is tested on Pytorch 1.8.2 LTS, and other versions might be incompatible.

In [ ]:
# Uninstall existing Pytorch on Colab, which might be incompatible or buggy.
!pip uninstall --yes torch torchvision torchaudio torchtext
# Install Pytorch 1.8.2 LTS (Long Term Support) and auto_LiRPA. It might take a few minutes depending on network speed.
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install git+https://github.com/KaidiXu/auto_LiRPA.git
# Clear installation output to avoid clutter.
from IPython.display import clear_output
clear_output()

Common Pytorch imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Imports for using auto_LiRPA

In [ ]:
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *

## Define the Computation (Neural Network)
To begin with, we define a **18-layer ResNet** using Pytorch. The network is defined as a standard nn.module object in Pytorch, and consists of **convolutional**, **pooling** and **batch normalization** layers. We will use our auto_LiRPA library to compute bounds for this network automatically, without manual derivations of the bounds.

In [ ]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, in_planes=64):
        super(ResNet, self).__init__()
        self.in_planes = in_planes

        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.layer1 = self._make_layer(block, in_planes, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, in_planes * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, in_planes * 4, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, in_planes * 8, num_blocks[3], stride=2)
        self.linear = nn.Linear(in_planes * 8 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(in_planes=2):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_planes=in_planes)

Now we create the model, and load some pretrained parameters for demonstration. Note that this pretrained model was naturally trained so only verifiable under small perturbations.

In [ ]:
model = ResNet18()
# Download the model
!wget -O resnet18_demo.pth http://download.huan-zhang.com/models/auto_lirpa/resnet18_natural.pth
# Load pretrained weights. This pretrained model is for illustration only; it
# does not represent state-of-the-art classification performance.
checkpoint = torch.load("resnet18_demo.pth")
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model = model.cuda()

--2022-02-23 17:51:07--  http://download.huan-zhang.com/models/auto_lirpa/resnet18_natural.pth
Resolving download.huan-zhang.com (download.huan-zhang.com)... 104.21.96.11, 172.67.171.242, 2606:4700:3030::6815:600b, ...
Connecting to download.huan-zhang.com (download.huan-zhang.com)|104.21.96.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208801 (204K)
Saving to: ‘resnet18_demo.pth’

resnet18_demo.pth   100%[===================>] 203.91K  --.-KB/s    in 0.1s    

2022-02-23 17:51:08 (1.61 MB/s) - ‘resnet18_demo.pth’ saved [208801/208801]



## Load dataset

We simply use the standard CIFAR-10 dataset. We load a random image from the dataset for demonstrating the usage of our framework.

In [ ]:
test_data = datasets.CIFAR10(
    "./data", train=False, download=True, 
    transform=transforms.Compose([transforms.ToTensor(), 
                                  transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])]))
# Choose one image from the dataset.
idx = 123
image = test_data[idx][0].view(1,3,32,32).cuda()
label = data = test_data[idx][1]
print('Ground-truth label:', label)
print('Model prediction:', model(image))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Ground-truth label: 2
Model prediction: tensor([[-2.1683, -6.2335,  5.0832, -2.8249, -3.9203, -2.3359, -2.0199, -3.7470,
         -7.4981, -5.9163]], device='cuda:0', grad_fn=<AddmmBackward>)


## Use `auto_LiRPA` to obtain provable lower and outer bounds under perturbation

There are three essential steps to use `auto_LiRPA`:

1.   Wrap a predefined computation in a `nn.Module` object with `auto_LiRPA.BoundedModule`;
2.   Define perturbation as a `BoundedTensor` (or `BoundedParameter` if you are perturbing model weights);
3.   Use the `compute_bounds()` method to obtain lower and upper bounds of the computational graph defined in `nn.Module`.



In [ ]:
# Step 1: wrap model with BoundedModule. 
bounded_model = BoundedModule(model=model, global_input=torch.zeros_like(image))
bounded_model.eval()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2113: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size_prods == 1:
/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:680: UserWarning: ONNX export mode is set to inference mode, but operator batch_norm is set to training  mode. The model will be exported in inference, as specified by the export mode.
  training_mode + ", as specified by the export mode.")


In [ ]:
# Step 2: define perturbation. Here we use a Linf perturbation on input image.
eps = 0.003
norm = np.inf
ptb = PerturbationLpNorm(norm=norm, eps=eps)
# Input tensor is wrapped in a BoundedTensor object.
bounded_image = BoundedTensor(x=image, ptb=ptb)
# We can use BoundedTensor to get model prediction as usual. Regular forward/backward propagation is unaffected.
print('Model prediction:', bounded_model(bounded_image))

Model prediction: tensor([[-2.1683, -6.2335,  5.0832, -2.8249, -3.9203, -2.3359, -2.0199, -3.7470,
         -7.4981, -5.9163]], device='cuda:0', grad_fn=<AddBackward0>)


As you can see above, the `BoundedModule` object wrapped by `auto_LiRPA` can be used the same way as a regular Pytorch model, with a `BoundedTensor` as its input.

In [ ]:
# Step 3: compute bounds using the compute_bounds() method.
print('Bounding method: backward (CROWN, DeepPoly)')
with torch.no_grad():  # If gradients of the bounds are not needed, we can use no_grad to save memory.
  lb, ub = bounded_model.compute_bounds(x=(bounded_image,), method='CROWN')

# Auxillary function to print bounds.
def print_bounds(lb, ub):
    lb = lb.detach().cpu().numpy()
    ub = ub.detach().cpu().numpy()
    for j in range(10):
        print("f_{j}(x_0): {l:8.3f} <= f_{j}(x_0+delta) <= {u:8.3f}".format(
            j=j, l=lb[0][j], u=ub[0][j], r=ub[0][j] - lb[0][j]))

print_bounds(lb, ub)

Bounding method: backward (CROWN, DeepPoly)
f_0(x_0):   -5.638 <= f_0(x_0+delta) <=    0.521
f_1(x_0):  -10.532 <= f_1(x_0+delta) <=   -2.419
f_2(x_0):    1.883 <= f_2(x_0+delta) <=    7.537
f_3(x_0):   -5.327 <= f_3(x_0+delta) <=   -0.827
f_4(x_0):   -7.217 <= f_4(x_0+delta) <=   -1.037
f_5(x_0):   -5.238 <= f_5(x_0+delta) <=   -0.151
f_6(x_0):   -5.686 <= f_6(x_0+delta) <=    0.118
f_7(x_0):   -7.934 <= f_7(x_0+delta) <=   -0.303
f_8(x_0):  -12.044 <= f_8(x_0+delta) <=   -3.793
f_9(x_0):   -9.329 <= f_9(x_0+delta) <=   -3.074


The backward mode perturbation analysis (an extension of [CROWN](https://https://arxiv.org/pdf/1811.00866.pdf)) provides relatively tight bounds. In this example above, the ground-truth label is 2. You can see that the model logit output for label 2 is bounded between 1.883 and 7.537, and we can guarantee that its the top-1 label under perturbation.

Next, we will compute the bounds using interval bound propagation (IBP), a previous approach that can also operate on general computational graphs. However, it produces much looser and vacuous bounds:

In [ ]:
# Our library also supports the interval bound propagation (IBP) based bounds, 
# but it produces much looser bounds.
print('Bounding method: IBP')
with torch.no_grad():
  lb, ub = bounded_model.compute_bounds(x=(bounded_image,), method='IBP')

print_bounds(lb, ub)

Bounding method: IBP
f_0(x_0): -23917152.000 <= f_0(x_0+delta) <= 14821585.000
f_1(x_0): -25477740.000 <= f_1(x_0+delta) <= 16557554.000
f_2(x_0): -18018624.000 <= f_2(x_0+delta) <= 13646834.000
f_3(x_0): -17182962.000 <= f_3(x_0+delta) <= 9431992.000
f_4(x_0): -22261390.000 <= f_4(x_0+delta) <= 12147498.000
f_5(x_0): -21668388.000 <= f_5(x_0+delta) <= 12951016.000
f_6(x_0): -24474524.000 <= f_6(x_0+delta) <= 11607180.000
f_7(x_0): -28624064.000 <= f_7(x_0+delta) <= 17297988.000
f_8(x_0): -29272032.000 <= f_8(x_0+delta) <= 17333456.000
f_9(x_0): -24436304.000 <= f_9(x_0+delta) <= 12459551.000


## Differentiability of our bounds

The bounds obtained by our `compute_bounds()` method are themselves differentiable w.r.t. input image or model parameters. We can obtain the gradients easily just as we usually do in Pytorch.  The gradients can be used for certified defense training. See our [training examples](https://github.com/KaidiXu/auto_LiRPA#basic-certified-training).


In [ ]:
bounded_model.zero_grad()
lb, ub = bounded_model.compute_bounds(x=(bounded_image,), method='CROWN')
# Create a dummy scalar function for demonstrating the differentiability.
loss = lb.sum()
loss.backward()
# This is the gradients of the loss w.r.t. first convolutional layer's weights:
print('grad norm:', list(model.modules())[1].weight.grad.norm(2))

grad norm: tensor(97.1450, device='cuda:0')


## More examples

We provide many examples of `auto_LiRPA` in our repository. You can find more details of these examples [here](https://github.com/KaidiXu/auto_LiRPA#more-working-examples). Notably, we provided the following examples for `auto_LiRPA`:

1. Certified defense on CIFAR-10, **TinyImageNet** and **ImageNet** (64*64) using large scale computer vision models such as DenseNet, ResNeXt and WideResNet.
2. Examples on using **loss fusion**, an efficient technique that scales linear relaxation based certified defense to large datasets, making certified defense training up to 1000 times faster compared to the previous approach.
3. Examples on training verifiably robust **LSTM** and **Transformer** models on natural language processing (**NLP**) tasks.
4. Examples on bounding network output given **model weight perturbations**. Existing frameworks can only handle perturbations on model inputs, not on model parameters (weights). This allows us to perform robustness verification or certified adversarial defense against weight perturbations. We can also train the bounds on model weights to obtain models with flat optimization landscapes. 